In [ ]:
#####################################################################
# This file is for doing whatever and probably shouldn't have been  #
# committed to Github, but there's an error in removing it so we    #
# have to live with it now, I guess.                                #
#####################################################################
from __future__ import print_function, division
import os
import sys

"""Open MP and MKL should speed up the time required to run these simulations!"""
# threads = sys.argv[1]
threads = 16
os.environ['NUMEXPR_MAX_THREADS'] = '{}'.format(threads)
os.environ['NUMEXPR_NUM_THREADS'] = '{}'.format(threads)
os.environ['OMP_NUM_THREADS'] = '{}'.format(threads)
os.environ['MKL_NUM_THREADS'] = '{}'.format(threads)

from quspin.operators import hamiltonian, exp_op, quantum_operator  # operators
from quspin.basis import spinful_fermion_basis_1d  # Hilbert space basis
from quspin.tools.measurements import obs_vs_time  # calculating dynamics
from quspin.tools.evolution import evolve  # evolving system
import numpy as np  # general math functions
from time import time  # tool for calculating computation time
from tqdm import tqdm
import matplotlib.pyplot as plt  # plotting library
from scipy.signal.windows import blackman
from scipy import fftpack
from scipy.interpolate import UnivariateSpline
from quspin.tools.measurements import project_op
from tools import HubbardModel as fhmodel, InitializeArchive
import psutil

In [ ]:
########################################################################################################################
# Declare Parameters of system to be loaded
########################################################################################################################

"""Parameters for a target or reference field"""
# Hubbard model
L = 10
t0 = 0.52
U = 0.05 * t0    # interaction strength

load_params = dict(
    angular_frequency=32.9,    # Field angular frequency THz
    field_amplitude=4.5,       # Field amplitude MV/cm
    lattice_constant=4,        # Lattice constant Angstroms
    nx=L,                      # system size
    n_up=L // 2 + L % 2,       # number of fermions with spin up
    n_down=L // 2,             # number of fermions with spin down
    hopping=t0,                # hopping strength
    interaction=U,             # interaction strength
    pbc=True,
    chem_potential=0,
    cycles=10,                 # time in cycles of field frequency
    n_steps=4000,              # Number of steps for time resolution
    ny=0,                      # 1D simulations do not use y-axis
    soc=0,                     # No spin orbit coupling
    gamma=0,
    tracking=True,             # Are you loading a field for tracking
    int_track=0,               # If so, you need to list the U for
    scf=True,
)

load = fhmodel(**load_params)
start = 0.0
times, dt = np.linspace(start, load.stop, num=load.n_steps, endpoint=True, retstep=True)